In [4]:
from dataclasses import dataclass
from typing import Tuple

import aadc.evaluate_wrappers
import numpy as np
import numpy.typing as npt

import aadc
from aadc import VectorFunctionWithAD
from aadc.recording_ctx import record_kernel


@dataclass
class BlackScholesModel:
    sigma: float
    rfr: float
    s0: float
    
    def simulate(self, final_time: float, normal_samples: npt.NDArray[np.double]) -> Tuple[npt.NDArray[np.double], npt.NDArray[np.double]]:
        num_steps = normal_samples.shape[1]
        dt = final_time / num_steps
        time = aadc.array(np.zeros(num_steps))
        gbm = aadc.array(np.zeros_like(normal_samples))
        
        gbm[:, 0] = self.s0
        bm_curr = 0.
        
        for i in range(1, num_steps):
            time[i] = i * dt
            bm_curr = bm_curr + normal_samples[:, i] * np.sqrt(dt)
            gbm[:, i] = self.s0 * np.exp((self.rfr - self.sigma**2/2) * time[i] + self.sigma * bm_curr)
            
        return time, gbm
    

@dataclass
class DownAndOutCallPayoff:
    barrier: float
    strike: float
    
    def evaluate(self, paths: npt.NDArray[np.double]) -> npt.NDArray[np.double]:
        scaling_factor = 1e+1
        barrier_distances = (paths - self.barrier) * scaling_factor
        survival_probs = np.prod((1 + np.tanh(barrier_distances)) / 2, axis=1)
        call_payoff = np.maximum(paths[:, -1] - self.strike, 0.0)
        payoffs = survival_probs * call_payoff
        return payoffs

M = 500
N = 1000
RANDOM_SEED = 2137
SPOT = 100.0
STRIKE = 100.0
RISK_FREE_RATE = 0.05
EXPIRY = 1.0
VOLATILITY = 0.2
BARRIER = 90.0

rng = np.random.default_rng(RANDOM_SEED)
normal_samples = rng.standard_normal((1, M))
params_np = aadc.array([
    SPOT, 
    STRIKE, 
    BARRIER,
    EXPIRY, 
    RISK_FREE_RATE, 
    VOLATILITY
])

with record_kernel() as kernel:
    params = aadc.array(params_np)
    pin = params.mark_as_input()
    active_samples = aadc.array(normal_samples)
    asin = active_samples.mark_as_input_no_diff()

    model = BlackScholesModel(params[-1], params[-2], params[0])
    payoff = DownAndOutCallPayoff(params[2], params[1])
    times, paths = model.simulate(params[3], active_samples)
    price_undiscounted = payoff.evaluate(paths)
    pundout = price_undiscounted.mark_as_output()
    price_discounted = price_undiscounted*np.exp(-params[-2]*params[3])
    pout = price_discounted.mark_as_output()

normal_samples_eval = rng.standard_normal((N, M))


Running cmake --build & --install in /home/ocmob/dev/aadc/AADC-Python-Bindings/build
You are using evaluation version of AADC. Expire date is 20250201


In [5]:
calibration_vf = VectorFunctionWithAD(kernel, pin[4:], pout, batch_param_args=np.squeeze(asin), param_args=pin[:4], num_threads=12)
calibration_vf.set_params(params_np[:4])
calibration_vf.set_batch_params(normal_samples_eval)
values, grads = calibration_vf.evaluate(params_np[4:])

value_calib = values.mean()
dvalue_d_bs_params = grads.mean(axis=0).squeeze()

print(value_calib, dvalue_d_bs_params)

9.541824644245017 [59.25275641 14.62033378]


In [6]:
evaluation_vf = VectorFunctionWithAD(kernel, pin, pout, batch_param_args=np.squeeze(asin), num_threads=12)
evaluation_vf.set_batch_params(normal_samples_eval)
values, grads = evaluation_vf.evaluate(params_np)

value_eval = values.mean()
dvalue_dparams = grads.mean(axis=0).squeeze()

print(value_eval, dvalue_dparams)

9.541824644245017 [ 0.83005581 -0.42013721 -0.3493186   4.4246712  59.25275641 14.62033378]


In [7]:
inputs = {
    **{param: param_value for param, param_value in zip(pin, params_np)},
    **{sample: samples_row for sample, samples_row in zip(np.squeeze(asin), normal_samples_eval)}
}
request = {
    pout.item(): pin.tolist(),
    pundout.item(): pin.tolist()
}

workers = aadc.ThreadPool(12)
values, derivs = aadc.evaluate(kernel, request, inputs, workers)